# Shor's factoring algorithm

[Shor's factoring algorithm](https://arxiv.org/abs/2208.09964) is perhaps the most famous application of quantum computing. It promises to be a fast way to factorise numbers, and since many encryption schemes are based on the fact that multiplying large numbers together is relatively fast but finding the prime factors of a large number is much slower, a shortcut here may eventually break these encryption schemes.

We will see, however, that we have relatively little to worry about for the time being. [The State of Factoring on Quantum Computers](https://arxiv.org/abs/2410.14397) (by David Willsche et al.) is that "so far only very
small integers have been successfully factored with Shor’s algorithm on a digital quantum computer; note that there exist many claims of factoring larger integers on digital quantum computers, but the underlying experiments
often rely on a certain kind of oversimplification that makes them equivalent to coin flipping. Even for
N = 15, 21, 35, one can argue that the explicitly compiled quantum circuits might not have been found without
previous knowledge about the answer to the integer factorisation problem."

[Shor's Factoring Algorithm and Modular Exponentiation Operators](https://arxiv.org/abs/2306.09122) by Robert L. Singleton Jr. for "a pedagogical presentation of Shor’s factoring algorithm" explains the modular exponentiation operators and includes example `qiskit` code. In particular there is a discussion on how to factorize numbers other than 15. Stéphane Beauregard has presented a [Circuit for Shor's Algorithm using 2n+3 Qubits](https://arxiv.org/abs/quant-ph/0205095).

[How Quantum Computers Break The Internet... Starting Now](https://youtu.be/-UrdExQW0cs) - YouTube video from Veritasium (March 2023).

Some comments on the feasibility of factoring large numbers with quantum computers, in terms of how many qubits would be required and for how long, are given in [Surface codes: Towards practical large-scale quantum computation](https://arxiv.org/abs/1208.0928) by Austin G. Fowler et. al. https://link.aps.org/doi/10.1103/PhysRevA.86.032324

This algorithm will make use of the QFT and phase estimation algorithms. 

If we chose some small integer $x$, we can define a function $f(p) = x^p$ where $p$ is also an integer. For example, for $x=7$ we would get

| _p_   | _f(p)_   |
| -: | -:    |
| 1	  | 7      |
| 2	  | 49     |
| 3	  | 343    |  
| 4	  | 2401   |
| 5	  | 16807  |
| 6   |	117649 |

and so on. However, now we redefine $f(p,k)$ to find the remainder of $x^p$ when we divide it by another integer, $k$, which is the product of two primes, $k = k_1 k_2$:
$$f(p,k) = x^p \mod k$$
Choosing $k = 15$ for example, for $p=1$, $f(1)=7$ so $15 \mod 7$ is just $7$. For $p=2$, $f(2)=49$ which is $3\times15$ with remainder 4, so $f(2,15) = 49 \mod 15 = 4$. This gives:

In [ ]:
x = 7
k = 15
print("p\tf(p)\tf(p,k)")
for p in range(0,14):
    fp=x**p
    fpk=fp % k
    print("{}\t{}\t{}".format(p,fp,fpk))

| _p_ | _f(p)_ | _f(p,k)_ |
| --- | ---  | --- |
| 0 | 1 | 1 |
| 1 | 7 | 7 |
| 2 | 49 | 4 |
| 3 | 343 | 13 |
| 4 | 2401 | 1 |
| 5 | 16807 | 7 |
| 6 | 117649 | 4 |
| 7 | 823543 | 13 |
| 8 | 5764801 | 1 |
| 9 | 40353607 | 7 |
| 10 | 282475249 | 4 |
| 11 | 1977326743 | 13 |
| 12 | 13841287201 | 1 |
| 13 | 96889010407 | 7 |

We notice that the function $f(p,k)$ repeats itself with the same four values: 7, 4, 13, 1. The smallest $p$ for which $f(p,k)=1$ is $p=4$ i.e. this function has periodicity 4. We have found the $p$ for which $x^p - 1 = 0$. This can be factorized as $(x^\frac{p}{2} - 1)(x^\frac{p}{2} + 1) = 0$.

$p$ happens to be an even number, so we can find
$$x_1 = x^\frac{p}{2} - 1 = 7^2 - 1 = 48$$
$$x_2 = x^\frac{p}{2} + 1 = 7^2 + 1 = 50$$
And furthermore, $x_1 x_2 = 2400$ is exactly divisible by $k = 15$. But since $x_2 - x_1 = 2$ they have no factor in common which is greater than $2$.

The [greatest common divisor](https://en.wikipedia.org/wiki/Euclidean_algorithm) between $x_1$ and $k$, $\gcd(x_1,k) = 3$, and $\gcd(x_2,k) = 5$. So $15 = 3 \times 5$.

We can also note that 

$1 \times 7 = 7,$ then $7 \mod 15 = 7$

$7 \times 7 = 49,$ then $49 \mod 15 = 4$

$4 \times 7 = 28,$ then $28 \mod 15 = 13$

$13 \times 7 = 91,$ then $91 \mod 15 = 1$

and so on.

## Classical computational implementation

We can perform the algorithm with some conventional computer code.

In [ ]:
import numpy as np
import math
from numpy import pi

#import matplotlib.pyplot as plt

k = 3*5 # number to factorize
for x in range(2,k):
    if math.gcd(x,k)==1:
        #print('try x = {}'.format(x))
        #x = 15 # pick an integer < k

        i = 0
        Ai = x % k # value for i = 1
        while Ai>1:
            i += 1
            Ai = x**i % k
            # print(i,Ai)

            p = i # print(p)
            if(p % 2 == 0):
                xp = x**int(p/2)
                x1 = int(xp-1)
                x2 = int(xp+1)
                
                k1 = math.gcd(x1,k)
                k2 = math.gcd(x2,k)
                
                if(k1 != 1 and k2 != 1 and k1 != k2):
                    #print(p,Ai)
                    #print(x1,x2)
                    print('{} = {}x{} from p = {} using x = {}'.format(k,k1,k2,p,x))

A key part of this algorithm is identifying the periodicity of the function $f(p,k) = x^p \mod k$.

## Quantum computing implementation

Define a unitary operator $$ U|p\rangle \equiv |x^p \bmod k \rangle $$

Starting in the state $|1\rangle$, for $x = 7$ and $k = 15$, $$U|1\rangle \equiv |7^1 \bmod 15\rangle = |7\rangle$$

$$U|7\rangle = U^2|1\rangle = |4\rangle$$
$$U|4\rangle = U^3|1\rangle = |13\rangle$$
$$U|13\rangle = U^4|1\rangle = |1\rangle$$

In general, we will eventually obtain $U^r|1\rangle = |1\rangle$; in this case, $r=4$. If $U$ comes all the way back around to $|1\rangle$ every $r$ operations, it means that we can consider that each operation of $U$ adds a phase of $2\pi/r$. In general, $U^r$ could actually have accumulated a phase of $2\pi s$ for some integer $s$ and still be back at $|1\rangle$, so if we do quantum phase estimation on $U$ we should find $\phi = s/r$ and then we can try to find which two integers $s$ and $r$ would give that $\phi$.

But first we need a way to implement the modular exponentiation in binary so that we can perform it on qubits. The mod 15 operation is easy to perform in binary - we just limit ourselves to 4 qubits. The will naturally make it easier to check if a number of the form $2^n - 1$ is a [Mersenne prime](https://en.wikipedia.org/wiki/Mersenne_prime). So we can achieve what we need by being able to multiply by $x=7$, or rather by being able to perform a controlled multiply-by-7 on the register containing $|p\rangle$ $2^n$ times according to each control qubit $n$. The control qubits will be initialized to $|+\rangle$ states, as for quantum phase estimation.


# Getting started

In a jupyter python notebook, ctrl-enter will run a python code block. Generally they should be self-contained although some will only make sense if they are run in order. "a" will add a block above, "b" will add a block below, "m" will convert a block from python code to "markup" (i.e. text) although you can also write $\LaTeX$ in a markup box. "dd" deletes a block. "y" converts a block back to code.

See http://localhost:8888/view/000-installing-qiskit.html for installation instructions.

API references:

https://docs.quantum.ibm.com/api/qiskit

https://docs.quantum.ibm.com/api/qiskit-ibm-runtime

https://docs.quantum.ibm.com/api/qiskit-ibm-provider

This notebook has been updated for ```qiskit 2.2```

You should also make sure you have the file ```vector_to_latex.py``` in the same folder

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_bloch_vector, plot_bloch_multivector, plot_distribution, plot_histogram, array_to_latex
from qiskit.result import Result
from qiskit.quantum_info import Statevector, Operator
from qiskit_aer import Aer
from math import sqrt, pi

### use vector_to_latex code from qiskit 0.44
from vector_to_latex import *

In [ ]:
x = 7
power = 1
"""multiplication by x mod 15"""
if x not in [2,4,7,8,11,13]:
    raise ValueError("'k' must be 2,4,7,8,11 or 13")
U = QuantumCircuit(4)        
for iteration in range(power):
    if x in [2,13]:
        U.swap(0,1)
        U.swap(1,2)
        U.swap(2,3)
    if x in [7,8]:
        U.swap(2,3)
        U.swap(1,2)
        U.swap(0,1)
    if x in [4, 11]:
        U.swap(1,3)
        U.swap(0,2)
    if x in [7,11,13]:
        for q in range(4):
            U.x(q)
            
U.draw()

We can watch what happens as we repeatedly apply this, but note that the qubit ordering is reversed such that state $1 = |1000\rangle$.

Also note that this generates the gates necessary to do the $x^p \mod k$ in which we can specify $x$, but the number we want to factorize is $k$, and this is hard-coded in. There does not seem to be a general algorithm for generating $x^p \mod k$ for an arbitrary $k$, especially a $k$ which is does not correspond to to a number $2^n - 1$ (for integer $n$) for which the ${} \mod{}$ operation is more natural. 

```qiskit``` is used on larger numbers in https://arxiv.org/abs/2306.09122 

In [ ]:
p=1
qc = QuantumCircuit(4)
qc.x(3)
qc.barrier()
if p>0:
    for i in range(0,p):
        qc.append(U,range(4))
        qc.barrier()

display(qc.draw())
#display(qc.decompose().draw())
state = Statevector(qc)
output=0
for bit in range(0,2**4):
    if state.data[bit]==1:
        output=bit

display(state.draw(output = 'latex'))
# print("output = |{}>".format(output))

In this ordering, the outputs are $$|1110\rangle =  7$$ $$|0010\rangle =  4$$ $$|1011\rangle =  13$$ $$|1000\rangle =  1$$

We can set this up as a controlled Unitary, which can be appended onto our quantum circuit:

In [ ]:
def c_kmod15(x, power):
    """Controlled multiplication by a mod 15"""
    if x not in [2,7,8,11,13]:
        raise ValueError("'x' must be 2,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(power):
        if x in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if x in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if x == 11:
            U.swap(1,3)
            U.swap(0,2)
        if x in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (x, power)
    c_U = U.control()
    return c_U

The controlled Unitaries making up the $x^p \mod k$ look like this:

In [ ]:
n_count = 8 # number of counting qubits
k = 7

qcexp = QuantumCircuit(n_count + 4, n_count)

q=0
qcexp.append(c_kmod15(k, 2**q), [q] + [i+n_count for i in range(4)])

qcexp.decompose().draw()

We can see that $SWAP$ operations, implemented as a combination of three $CX$ operations in the IBM hardware, become three Toffoli gates with the extra control corresponding to the relevant counting qubit. The $X$ gates become $CX$.

And also define the inverse QFT:

In [ ]:
def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

This is what the inverse QFT looks like:

In [ ]:
qubits=4
qcqft = QuantumCircuit(qubits)
qcqft.append(qft_dagger(qubits), range(qubits))
qcqft.decompose().draw()

In [ ]:
n_count = 4 # number of counting qubits
k = 7

# Create QuantumCircuit with n_count counting qubits
# plus 4 qubits for U to act on
qc = QuantumCircuit(n_count + 4, n_count)

# Initialise counting qubits in state |+>
for q in range(n_count):
    qc.h(q)
    
# And auxiliary register in state |1>
qc.x(3+n_count) # qc.x(0+n_count)

# Do controlled-U operations
for q in range(n_count):
    qc.append(c_kmod15(k, 2**q), 
             [q] + [i+n_count for i in range(4)])

# Do inverse-QFT
qc.append(qft_dagger(n_count), range(n_count))

# Measure circuit
qc.measure(range(n_count), range(n_count))
qc.draw(fold=-1)  # -1 means 'do not fold' 

In [ ]:
qasm_sim = Aer.get_backend('qasm_simulator')
t_qc = transpile(qc, qasm_sim)
results = qasm_sim.run(t_qc, shots=1000).result()
counts = results.get_counts()
plot_histogram(counts)

Since we have 8 qubits, these results correspond to measured phases of:

In [ ]:
import pandas as pd

rows, measured_phases = [], []
for output in counts:
    decimal = int(output, 2)  # Convert (base 2) string to decimal
    phase = decimal/(2**n_count)  # Find corresponding eigenvalue
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append([f"{output}(bin) = {decimal:>3}(dec)", 
                 f"{decimal}/{2**n_count} = {phase:.2f}"])
# Print the rows in a table
headers=["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)

So $\phi = s/r$ could be $0$, $1/4$, $1/2$, or $3/4$. So two of these answers give the correct answer of $r=4$ but we note that we cannot identify $2/4$ instead of $1/2$. Having found $r$ we do continue with classical digital post-processing.

In [ ]:
r = 4

k = 15 # the number we wanted to factorize
if(r % 2 == 0):
    xr = x**int(r/2)
    x1 = int(xr-1)
    x2 = int(xr+1)

    k1 = math.gcd(x1,k)
    k2 = math.gcd(x2,k)

    if(k1 != 1 and k2 != 1 and k1 != k2):
        #print(p,Ai)
        #print(x1,x2)
        print('{} = {}x{} from p = {} using x = {}'.format(k,k1,k2,r,x))

We can quickly check that this is the right answer.

We gain an advantage from the efficiency of the QFT† but the exponentiation steps are the bottleneck. Efficient (but perhaps even less intuitive) steps to perform modular exponentiation for any number are being saught. [As of 2019, 21 has been factorized but 35 has not](https://arxiv.org/abs/1903.00768). Larger numbers have been factorized by brute-force checking but Shor's algorithm is expected to perform better once enough error-free qubits are available to reach higher numbers.

Here we present the circuit without classical registers so that we can see the Bloch spheres:

In [ ]:
n_count = 4 # number of counting qubits
k = 7 # 13 # 7

# Create QuantumCircuit with n_count counting qubits
# plus 4 qubits for U to act on
qc = QuantumCircuit(n_count + 4)

# Initialise counting qubits in state |+>
for q in range(n_count):
    qc.h(q)
    
# And auxiliary register in state |1>
qc.x(3+n_count) # qc.x(0+n_count)

# Do controlled-U operations
for q in range(n_count):
    qc.append(c_kmod15(k, 2**q), 
             [q] + [i+n_count for i in range(4)])


display(qc.draw())
state = Statevector(qc)
display(state.draw(output = 'latex'))
#display(Math(vector_to_latex(state)))
display(state.draw(output = 'bloch'))

In [ ]:
# Do inverse-QFT
qc.append(qft_dagger(n_count), range(n_count))
qc.draw()
state = Statevector(qc)
display(state.draw(output = 'latex'))
#display(Math(vector_to_latex(state)))
display(state.draw(output = 'bloch'))

In [ ]:
import time
factors = False
k = 35 # number to factorize
for x in range(2,k):
    starttime = time.time()
    if math.gcd(x,k)==1:
        #print('try x = {}'.format(x))
        #x = 15 # pick an integer < k

        i = 0
        Ai = x % k # value for i = 1
        while Ai>1:
            i += 1
            Ai = x**i % k
            #print(i,Ai)

            p = i # print(p)
            if(p % 2 == 0):
                xp = x**int(p/2)
                x1 = int(xp-1)
                x2 = int(xp+1)
                
                k1 = math.gcd(x1,k)
                k2 = math.gcd(x2,k)
                
                if(k1 != 1 and k2 != 1 and k1 != k2):
                    #print(p,Ai)
                    #print(x1,x2)
                    factors = True
                    endtime = time.time()
                    elapsed = endtime - starttime
                    print('{} = {}x{} from p = {} using x = {} in {:.2f} microseconds'.format(k,k1,k2,p,x,1e6*elapsed))
                
    elif math.gcd(x,k)>1:
        foundgcd=math.gcd(x,k)
        factors = True
        endtime = time.time()
        elapsed = endtime - starttime
        if x==foundgcd:
            print('{} = {}x{:.0f} was found in {:.2f} microseconds because {} is directly divisible by {}'.format(k,foundgcd,k/foundgcd, 1e6*elapsed, k, foundgcd))            

        else:
            print('{} = {}x{:.0f} was found in {:.2f} microseconds because {} has the common factor {} with {}'.format(k,foundgcd,k/foundgcd, 1e6*elapsed, x, foundgcd, k))
            
if factors == False:
    endtime = time.time()
    elapsed = endtime - starttime
    print('no factors found for {} in {:.3f} microseconds'.format(k,1e6*elapsed))

In [ ]:
import time
factors = False
# factorize a Mersenne number
# k = 2047

# a prime number
# k = 2621

# a composite number
k = 41989

# check a 5-digit prime number
# k = 14951

# check a 6-digit prime number (might take about 30 seconds)
# k = 489061

# factorize a large number (might take 2-3 minutes)
# k = 8388607

# check a 7-digit prime number (might take hours)
# k = 4393139

for x in range(7,8):
    starttime = time.time()
    if math.gcd(x,k)==1:
        #print('try x = {}'.format(x))
        #x = 15 # pick an integer < k

        i = 0
        Ai = x % k # value for i = 1
        while Ai>1:
            i += 1
            Ai = x**i % k
            #print(i,Ai)

            p = i # print(p)
            if(p % 2 == 0):
                xp = x**int(p/2)
                x1 = int(xp-1)
                x2 = int(xp+1)
                
                k1 = math.gcd(x1,k)
                k2 = math.gcd(x2,k)
                
                if(k1 != 1 and k2 != 1 and k1 != k2):
                    #print(p,Ai)
                    #print(x1,x2)
                    endtime = time.time()
                    elapsed = endtime - starttime
                    factors = True
                    if elapsed < 1e-3:
                        print('{} = {}x{} from p = {} using x = {} in {:.3f} microseconds'.format(k,k1,k2,p,x,1e6*elapsed))
                    
                    elif elapsed < 1.0: 
                        print('{} = {}x{} from p = {} using x = {} in {:.3f} milliseconds'.format(k,k1,k2,p,x,1e3*elapsed))
                    
                    else:
                        print('{} = {}x{} from p = {} using x = {} in {:.3f} seconds'.format(k,k1,k2,p,x,elapsed))

    elif math.gcd(x,k)>1:
        foundgcd=math.gcd(x,k)
        factors = True
        endtime = time.time()
        elapsed = endtime - starttime
        if x==foundgcd:
            print('{} = {}x{:.0f} was found in {:.2f} microseconds because {} is directly divisible by {}'.format(k,foundgcd,k/foundgcd, 1e6*elapsed, k, foundgcd))            

        else:
            print('{} = {}x{:.0f} was found in {:.2f} microseconds because {} has the common factor {} with {}'.format(k,foundgcd, 1e6*elapsed, k/foundgcd, x, foundgcd, k))

if factors == False:
    endtime = time.time()
    elapsed = endtime - starttime
    if elapsed < 1e-3:
        print('no factors found for {} in {:.3f} microseconds'.format(k,1e6*elapsed))

    elif elapsed < 1.0: 
        print('no factors found for {} in {:.3f} milliseconds'.format(k,1e3*elapsed))

    else:
        print('no factors found for {} in {:.3f} seconds'.format(k,elapsed))
    
        

In [ ]:
n_count = 4 # number of counting qubits
k = 7 # 13 or 7

# Create QuantumCircuit with n_count counting qubits
# plus 4 qubits for U to act on
# qc = QuantumCircuit(n_count + 4, n_count)
qc = QuantumCircuit(n_count + 4)

# Initialise counting qubits in state |+>
#for q in range(n_count):
#    qc.h(q)
    
# And auxiliary register in state |1>
qc.x(3+n_count) # qc.x(0+n_count)

# Do controlled-U operations
for q in range(n_count):
    qc.append(c_kmod15(k, 2**q), 
             [q] + [i+n_count for i in range(4)])

# Do inverse-QFT
# qc.append(qft_dagger(n_count), range(n_count))

qc.draw()

## Transpilation to (Fake) Real IBM hardware

In [ ]:
from qiskit.visualization import plot_circuit_layout
from qiskit_ibm_runtime.fake_provider import FakeGeneva, FakeMelbourneV2

In [ ]:
n_count = 4 # number of counting qubits
k = 7 # 13 # 7

# Create QuantumCircuit with n_count counting qubits
# plus 4 qubits for U to act on
qc = QuantumCircuit(n_count + 4, n_count)

# Initialise counting qubits in state |+>
for q in range(n_count):
    qc.h(q)
    
# And auxiliary register in state |1>
qc.x(3+n_count) # qc.x(0+n_count)

# Do controlled-U operations
for q in range(n_count):
    qc.append(c_kmod15(k, 2**q), 
             [q] + [i+n_count for i in range(4)])

# Do inverse-QFT
qc.append(qft_dagger(n_count), range(n_count))

# Measure circuit
qc.measure(range(n_count), range(n_count))
# qc.draw(fold=-1)  # -1 means 'do not fold' 

backend = FakeMelbourneV2()
new_qc = transpile(qc, backend=backend, optimization_level=0)
display(plot_circuit_layout(new_qc, backend))
display(new_qc.draw(fold=-1))

In [ ]:
qasm_sim = Aer.get_backend('qasm_simulator')
results = qasm_sim.run(new_qc, shots=10).result()
counts = results.get_counts()
display(plot_histogram(counts))

# import pandas as pd

rows, measured_phases = [], []
for output in counts:
    decimal = int(output, 2)  # Convert (base 2) string to decimal
    phase = decimal/(2**n_count)  # Find corresponding eigenvalue
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append([f"{output}(bin) = {decimal:>3}(dec)", 
                 f"{decimal}/{2**n_count} = {phase:.2f}"])
# Print the rows in a table
headers=["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)

In [ ]:
import time
from sys import modules
from IPython.display import HTML, display
import qiskit
from qiskit.utils import local_hardware_info
html = "<h3>Version Information</h3>"
html += "<table>"
html += "<tr><th>Software</th><th>Version</th></tr>"

packages = {"qiskit": qiskit.__version__}
qiskit_modules = {module.split(".")[0] for module in modules.keys() if "qiskit" in module}

for qiskit_module in qiskit_modules:
    packages[qiskit_module] = getattr(modules[qiskit_module], "__version__", None)

for name, version in packages.items():
    if version:
        html += f"<tr><td><code>{name}</code></td><td>{version}</td></tr>"

html += "<tr><td colspan='2'>%s</td></tr>" % time.strftime("%a %b %d %H:%M:%S %Y %Z")
html += "</table>"

display(HTML(html))